# 多线程

## 多线程的含义

首先需要了解什么是进程

进程：一个可以独立运行的程序单元。比如打开一个浏览器，就是打开了一个浏览器进程；但是一个进程同时可以处理很多事情，比如一个浏览器中可以打开多个页面，多个页面可以同时运行，互不干扰。

为什么可以同时运行多个任务呢？这就引出了线程的概念了，其实这一个个任务，就对应着一个个线程的执行。

而进程就是线程的集合，进程就是由多个线程构成的，线程是操作系统进行运算测度的最小单位，是进程中的一个最小运行单元。

多线程就是一个进程中同时执行多个线程，前面浏览器的情景就是典型的多线程执行。

## 并发和并行

说到多线程和多进程，还需要讲解一下并发和并行。

### 并发(concurrency)

同一时刻只能有一条指令执行，但是多个线程的对应指令被快速轮换地执行。比如一个处理器在线程A,B之间快速切换执行，但是执行和切换速度之快，人感知不到多个线程的轮换执行。所以**宏观上看起来多个线程在同时运行，但是微观上是处理器在不同的线程之间切换和执行**

### 并行(parallel)

同一时刻，有多条指令在多个处理器上同时执行，并行必须依赖多个处理器。如果我们的计算机处理器只有一个核，那它只能通过并发的方式运行这些线程。如果有多个核，当一个核在执行一个线程时，另一个核可以执行另一个线程，这样两个线程就实现了并行运行。

## 多线程适用场景

在一个程序执行过程中，有些操作时比较耗时和需要等待的（例如等待数据库的查询结果），这时如果使用单线程，处理器必须等到这些操作完成后才能继续向下执行其他操作。如果使用多线程，就可以在等待某个线程时，去执行其他线程，从而整体上提高执行效率。

网络爬虫也是一个经典的需要多线程的例子：爬虫在向服务器发送请求后，需要等待服务器的响应，这种IO密集型任务可以启用多线程，这样可以在等待某个线程时去处理其他任务。从而大大提高整体的爬取效率。

还有一种任务叫做计算密集型，也就是CPU密集型任务，任务的运行一直需要处理器的参与，那么使用多线程反而会增加切换线程的时间，整体效率反而变低。

## Python实现多线程

Python实现多线程的模块叫做threading。

### thread直接创建多线程

In [9]:
import threading
import time 

def target(second):
    # 线程的名字就是通过threading.current_thread().name获得，主线程的名字为MainThreat，子线程的名字为Thread-num
    print(f'threat {threading.current_thread().name} is running')
    print(f'threat {threading.current_thread().name} sleep {second}s')
    time.sleep(second)
    print(f'threat {threading.current_thread().name} is ended')

print(f'Threading {threading.current_thread().name} is running')
for i in [1,5]:
    # 使用threading.Tread()类新建了两个线程，target参数就是刚才定义的target函数，args通过列表的形式传递
    thread = threading.Thread(target=target,args=[i])
    # 新建线程声明之后，使用start()方法即可开始线程的运行
    thread.start()
print(f'Threading {threading.current_thread().name} is ended')

Threading MainThread is running
threat Thread-18 is running
threat Thread-18 sleep 1s
threat Thread-19 is running
threat Thread-19 sleep 5sThreading MainThread is ended

threat Thread-18 is ended
threat Thread-19 is ended


Threading MainThread is running

threat Thread-1 is running

threat Thread-1 sleep 1s

threat Thread-2 is running

threat Thread-2 sleep 5s

Threading MainThread is ended

threat Thread-1 is ended

threat Thread-2 is ended

上面是代码运行应该得到的输出（在jupyter看不到threat1、2的end？）。可以看出，主线程首先结束，接着Threat-1和Threat-2才结束，并且分别间隔了1s和4s。说明主线程并没有等待子线程运行完毕才结束，而是直接结束。

如果我们想主线程等待子线程运行完毕才推出，可以让每个子线程对线都调用join方法，实现如下：

In [10]:
threads = []
for i in [1,5]:
    thread = threading.Thread(target=target,args=[i])
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()

threat Thread-20 is running
threat Thread-20 sleep 1s
threat Thread-21 is running
threat Thread-21 sleep 5s
threat Thread-20 is ended
threat Thread-21 is ended


## 继承Thread类创建子线程

通过继承Thread类的方式创建一个线程，该线程需要执行的方法写在类的run方法里面即可。上面的例子可等价写成：



In [11]:
import threading
import time 

class MyThread(threading.Thread):
    def __init__(self,second):
        threading.Thread.__init__(self)
        self.second = second
    
    def run(self):
        print(f'Threading {threading.current_thread().name} is running')
        print(f'Threading {threading.current_thread().name} sleep {self.second}s')
        time.sleep(self.second)
        print(f'Threading {threading.current_thread().name} is ended')

print(f'Threading {threading.current_thread().name} is running')
threads=[]
for i in [1,5]:
    thread = MyThread(i)
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
print(f'Threading {threading.current_thread().name} is ended')

Threading MainThread is running
Threading Thread-22 is running
Threading Thread-22 sleep 1s
Threading Thread-23 is running
Threading Thread-23 sleep 5s
Threading Thread-22 is ended
Threading Thread-23 is ended
Threading MainThread is ended


## 守护线程

如果一个线程被设置为守护线程，那么意味着这个线程是“不重要”的，如果主线程结束了而该守护线程还没有运行完，那么它将会被强制结束

In [12]:
print(f'Threading {threading.current_thread().name} is running')
t1 = MyThread(1)
t1.start()
t2 = MyThread(5)
# 将threat2设置为守护线程 setDaemon
t2.setDaemon(True)
t2.start()
print(f'Threading {threading.current_thread().name} is ended')

Threading MainThread is running
Threading Thread-24 is running
Threading Thread-24 sleep 1s
Threading Thread-25 is running
Threading Thread-25 sleep 5s
Threading MainThread is ended


在这里我们通过 setDaemon 方法将 t2 设置为了守护线程，这样主线程在运行完毕时，t2 线程会随着线程的结束而结束。

这里并没有调用 join 方法，如果我们让 t1 和 t2 都调用 join 方法，主线程就会仍然等待各个子线程执行完毕再退出，不论其是否是守护线程。

## 互斥锁

在一个进程中，多个线程是共享资源的，比如在一个进程中，有一个全局变量count，现在我们声明多个线程，每个线程运行时都给count加一。


In [17]:
count = 0
class MyThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)

    def run(self):
        global count
        temp = count + 1
        time.sleep(0.001)
        count = temp 

threads = []
for _ in range(1000):
    thread = MyThread()
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()
print(f'Final count: {count}')

Final count: 20


因为 count 这个值是共享的，每个线程都可以在执行 temp = count 这行代码时拿到当前 count 的值，但是这些线程中的一些线程可能是并发或者并行执行的，这就导致不同的线程拿到的可能是同一个 count 值，最后导致有些线程的 count 的加 1 操作并没有生效，导致最后的结果偏小。

所以，如果多个线程同时对某个数据进行读取或修改，就会出现不可预料的结果。为了避免这种情况，我们需要对多个线程进行同步，要实现同步，我们可以对需要操作的数据进行加锁保护，这里就需要用到 threading.Lock 了。

加锁保护是什么意思呢？就是说，某个线程在对数据进行操作前，需要先加锁，这样其他的线程发现被加锁了之后，就无法继续向下执行，会一直等待锁被释放，只有加锁的线程把锁释放了，其他的线程才能继续加锁并对数据做修改，修改完了再释放锁。这样可以确保同一时间只有一个线程操作数据，多个线程不会再同时读取和修改同一个数据，这样最后的运行结果就是对的了。

In [19]:
count = 0
class MyThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)

    def run(self):
        global count
        lock.acquire()
        temp = count + 1
        time.sleep(0.001)
        count = temp 
        lock.release()
        
lock = threading.Lock()
threads = []
for _ in range(1000):
    thread = MyThread()
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()
print(f'Final count: {count}')

Final count: 1000


关于 Python 多线程的内容，这里暂且先介绍这些，关于 theading 更多的使用方法，如信号量、队列等，可以参考官方文档：https://docs.python.org/zh-cn/3.7/library/threading.html#module-threading。

## Python 多线程的问题

由于 Python 中 GIL 的限制，导致不论是在单核还是多核条件下，在同一时刻只能运行一个线程，导致 Python 多线程无法发挥多核并行的优势。

GIL 全称为 Global Interpreter Lock，中文翻译为全局解释器锁，其最初设计是出于数据安全而考虑的。

在 Python 多线程下，每个线程的执行方式如下：

- 获取 GIL

- 执行对应线程的代码

- 释放 GIL

可见，某个线程想要执行，必须先拿到 GIL，我们可以把 GIL 看作是通行证，并且在一个 Python 进程中，GIL 只有一个。拿不到通行证的线程，就不允许执行。这样就会导致，即使是多核条件下，一个 Python 进程下的多个线程，同一时刻也只能执行一个线程。

不过对于爬虫这种 IO 密集型任务来说，这个问题影响并不大。而对于计算密集型任务来说，由于 GIL 的存在，多线程总体的运行效率相比可能反而比单线程更低。